In [ ]:
%pip install datasets

In [1]:
import json
import pandas as pd
import os
from sklearn.dummy import DummyClassifier
from datasets import load_dataset

/home/tajak/miniconda3/envs/conda_emma/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the test datasets

test_en = pd.read_json("../../datasets/ParlaSent-EN-test/ParlaSent_EN_test.jsonl", lines=True)
test_bcs = pd.read_json("../../datasets/ParlaSent-BSC-test/ParlaSent_BCS_test.jsonl", lines=True)

print(test_en.shape, test_bcs.shape)

(2600, 14) (2600, 14)


In [3]:
# Load the training dataset

train_df = pd.read_json("../../datasets/ParlaSent-train/ParlaSent_training.jsonl", lines=True)

print(train_df.shape)

train_df.head(2)

(13000, 16)


,text,country,annotator1,annotator2,reconciliation,labels,document_id,sentence_id,term,date,name,party,gender,birth_year,split,ruling
0,Da li je pošteno da se ukida prethodna stopa i...,HR,N_Neutral,Negative,M_Negative,Negative,17023,2536,9.0,2016-11-15,"Pupovac, Milorad",SDSS,M,1955.0,train,Opposition
1,Znam pouzdano da su među specijalnim snagama b...,SRB,Negative,Negative,Negative,Negative,14362,1648,9.0,2013-03-28,"Halimi, Riza",None,M,1947.0,train,None


In [4]:
def dummy(train_df, test_df, test_df_name):
    # Create X_train and Y_train parts, used for sci kit learning
    # List of texts in training split
    X_train = list(train_df.text)
    # List of labels in training split
    Y_train = list(train_df.labels)

    # List of texts in test split
    X_test = list(test_df.text)
    # List of labels in test split
    Y_test = list(test_df.labels)

    print(len(X_train), len(Y_train), len(X_test), len(Y_test))

    # Create a list of labels
    labels = list(test_df.labels.unique())
    print("Labels: {}".format(labels))

    for strategy in ["stratified", "most_frequent"]:
        model = f"dummy-{strategy}"

        dummy_mf = DummyClassifier(strategy=strategy)

        # Train the model
        dummy_mf.fit(X_train, Y_train)

        #Get the predictions
        y_pred_mf = dummy_mf.predict(X_test)

        y_pred = list(y_pred_mf)

        # Create a json with results
        current_results = {
            "system": model,
            "predictions": [
                {
                "train": "ParlaSent",
                "test": "{}".format(test_df_name),
                "predictions": y_pred,
                }
            ],
            #"model": model_type_dict[model][1],
            #"args": model_args,
            }

        # Save the results as a new json
        with open("submissions/submission-{}-{}.json".format(model, test_df_name), "w") as file:
            json.dump(current_results, file)

        print("Classification with {} on {} finished.".format(model, test_df_name))


In [5]:
dummy(train_df, test_en, "ParlaSent-EN-test")

13000 13000 2600 2600
Labels: ['Negative', 'Neutral', 'Positive']
Classification with dummy-stratified on ParlaSent-EN-test finished.
Classification with dummy-most_frequent on ParlaSent-EN-test finished.


In [6]:
dummy(train_df, test_bcs, "ParlaSent-BCS-test")

13000 13000 2600 2600
Labels: ['Negative', 'Neutral', 'Positive']
Classification with dummy-stratified on ParlaSent-BCS-test finished.
Classification with dummy-most_frequent on ParlaSent-BCS-test finished.
